# Using DwC-A_dotnet With Plotly.NET

This notebook shows a simple example of using Darwin Core Archive data with the [Plotly.NET](https://plotly.github.io/Plotly.NET/) charts.  Using LinQ we query all the occurrence data for a specific sampling event and display a column graph of totals for each genus found.

In [ ]:
#r "nuget: FSharp.Compiler.Service,*-*"
#r "nuget: System.CommandLine,*-*"
#r "nuget: Plotly.NET,2.0.0-beta6"
#r "nuget: Plotly.NET.Interactive,2.0.0-beta6"

In [ ]:
#r "nuget:DwC-A_dotnet,0.5.1"
#r "nuget:DwC-A_dotnet.Interactive,0.1.1-Pre"

In [4]:
using DwC_A;

var archive = new ArchiveReader(@"./data/dwca-rooftop-v1.4");

Change the number in the `.Skip(5)` line of the first query to retrieve a different eventID.  This ID will be displayed in the column chart title.

In [ ]:
using Plotly.NET;
using DwC_A.Terms;

var eventID = archive.CoreFile.DataRows
    .Skip(5)  //Change this number and run the cell again and to see the chart for a new eventID
    .Take(1)
    .First()[Terms.eventID];
var occurrences = archive.Extensions.GetFileReaderByFileName("occurrence.txt");

var dataSeries = occurrences.DataRows
    .Where(n => n[Terms.eventID] == eventID)
    .GroupBy(n => n[Terms.genus])
    .Select(g => 
        Tuple.Create(g.First()[Terms.genus],
            g.Sum(c => int.Parse(c[Terms.individualCount]))));

var chart = Plotly.NET.Chart.Column<string, int, int>(dataSeries);
var layout = new Plotly.NET.Layout();
chart.WithSize(1100, 500)
    .WithTitle(eventID);
chart